Find the medications that were prescribed by at least three different doctors.
Relevant DF: 
df1 = medications (medication_id, medication_name), 
df2 = prescriptions (prescription_id, doctor_id, medication_id)

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

medications_data = [
    (1, "Medication A"),
    (2, "Medication B"),
    (3, "Medication C"),
    (4, "Medication D"),
    (5, "Medication E")
]

prescriptions_data = [
    (1, 1, 1),
    (2, 2, 1),
    (3, 3, 1),
    (4, 1, 2),
    (5, 2, 2),
    (6, 3, 2),
    (7, 1, 3),
    (8, 2, 4),
    (9, 3, 4),
    (10, 4, 5),
    (11, 5, 5),
    (12, 6, 5)
]

spark= SparkSession.builder.getOrCreate()

medications_df = spark.createDataFrame(medications_data, ["medication_id", "medication_name"])
prescriptions_df = spark.createDataFrame(prescriptions_data, ["prescription_id", "doctor_id", "medication_id"])

In [3]:
medications_df.show()
prescriptions_df.show()

+-------------+---------------+
|medication_id|medication_name|
+-------------+---------------+
|            1|   Medication A|
|            2|   Medication B|
|            3|   Medication C|
|            4|   Medication D|
|            5|   Medication E|
+-------------+---------------+

+---------------+---------+-------------+
|prescription_id|doctor_id|medication_id|
+---------------+---------+-------------+
|              1|        1|            1|
|              2|        2|            1|
|              3|        3|            1|
|              4|        1|            2|
|              5|        2|            2|
|              6|        3|            2|
|              7|        1|            3|
|              8|        2|            4|
|              9|        3|            4|
|             10|        4|            5|
|             11|        5|            5|
|             12|        6|            5|
+---------------+---------+-------------+



In [9]:
prescriptions_df\
    .groupBy(col('medication_id')).agg(countDistinct('doctor_id').alias('c'))\
    .where('c>2')\
    .select('medication_id')\
    .join(medications_df,['medication_id'],'inner')\
    .show()

+-------------+---------------+
|medication_id|medication_name|
+-------------+---------------+
|            1|   Medication A|
|            2|   Medication B|
|            5|   Medication E|
+-------------+---------------+

